DiSQ is intended to be used in two ways:
- Using the engineering GUI to monitor the dish structure and run simple commands.
- Using the sculib and data logger in Python scripts for more complex testing.

Both methods require an OPCUA server to connect to.

In [ ]:
from disq import sculib, logger, hdf5_to_graph, hdf5_to_csv
from time import sleep
# Necessary for interactive graph window:
%matplotlib qt

In [ ]:
# Set up client
client = sculib.SCU(host="127.0.0.1", port=4840, endpoint="/OPCUA/SimpleServer", namespace="CETC54", username="LMC", password="lmc")
# Set up logger
data_logger = logger.Logger(high_level_library=client)
# Tell the logger what nodes to subscribe to
nodes = ["Azimuth.p_Act", "Elevation.p_Act"]
period = 100 # In milliseconds
data_logger.add_nodes(nodes, period)
# Start the data logging
data_logger.start()
# Run the test
#client.commands["CommandArbiter.TakeReleaseAuth"](False, "Tester")
#client.commands["CommandArbiter.TakeReleaseAuth"](True, "Tester")
AzElVel = [90.0, 15.0, 3.0, 1.0]
#AzElVel = [10.0, 45.0, 3.0, 1.0]
msg = client.commands["Management.Slew2AbsAzEl"](*AzElVel)
print(f"msg = {msg}")
#client.commands["Management.Stow"]
sleep(25)
data_logger.stop()
data_logger.wait_for_completion()
# Finished writing HDF5 file

In [ ]:
# Create the grapher object and get the file name from the data logger
grapher = hdf5_to_graph.Grapher()
file = data_logger.file

In [ ]:
# Check what nodes are in the file, and the file start and stop times
grapher.hdf5_info(file)

In [ ]:
# Produce the graph from the desired nodes
grapher.graph(file, *nodes)

In [ ]:
converter = hdf5_to_csv.Converter()
output_file = "results/" + data_logger.start_time.isoformat() + ".csv"
step = 100 # In milliseconds
converter.make_csv(file, output_file, nodes, step_ms=step)